In [9]:
from datasets import load_dataset

ds = load_dataset("Anthropic/hh-rlhf")

In [10]:
ds

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 160800
    })
    test: Dataset({
        features: ['chosen', 'rejected'],
        num_rows: 8552
    })
})

In [11]:
ds.save_to_disk("Anthropic-hh-rlhf")

Saving the dataset (1/1 shards): 100%|██████████| 8552/8552 [00:00<00:00, 445813.24 examples/s]


In [12]:
ds["train"].to_csv("train.csv")
ds["test"].to_csv("test.csv")


Creating CSV from Arrow format: 100%|██████████| 9/9 [00:00<00:00, 17.29ba/s]


16560280

In [ ]:
ds["train"].to_json("train.json")
ds["test"].to_json("test.json")


Creating json from Arrow format:   0%|          | 0/161 [00:00<?, ?ba/s]

Creating json from Arrow format: 100%|██████████| 9/9 [00:00<00:00, 43.41ba/s]


17136675

In [ ]:
# from typing import Optional
# from datasets import load_dataset, load_dataset_builder
# import logging

# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)


# class DatasetWrapper:
#     """Wrapper class for interacting with datasets, allowing for loading, filtering, and transforming data.
#     Attributies:
#     dataset_path (str)
#     dataset_name (str)
#     token (str): The API token for accessing the dataset
#     dataset (Dataset): The loaded dataset
#     """

#     def __init__(
#         self,
#         dataset_path: str,
#         dataset_name: str = "",
#         token: Optional[str] = None,
#     ) -> None:
#         """Initialize the wrapper
#         Args:
#             dataset_path (str),
#             dataset_name (str): the name of the dataset ie. subdirectory with specific version of dataset
#             token (str): The API token for accessing the dataset
#         """
#         self.dataset_path = dataset_path
#         self.dataset_name = dataset_name
#         self.token = token
#         self.dataset = self._load_dataset()

#     def _load_dataset(self, split="train"):
#         """Loads the dataset using the specified path and name."""
#         ds_builder = load_dataset_builder(
#             self.dataset_path, name=self.dataset_name, token=self.token
#         )
#         logger.info("Dataset description:", ds_builder.info.description)
#         logger.info("Dataset features:", ds_builder.info.features)
#         if self.token:
#             return load_dataset(
#                 self.dataset_path, self.dataset_name, split=split, token=self.token
#             )
#         else:
#             return load_dataset(self.dataset_path, self.dataset_name, split=split)

In [4]:
# wrapper = DatasetWrapper(dataset_path="Anthropic/hh-rlhf")